# Data Science Intern at Let's Grow More LGMVIP 

# More Advanced Level Task

# ML Facial recognition to detect mood and suggest songs accordingly

## Part1 - TRAINING

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Linraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_dir = '../input/fer2013/train'
test_dir = '../input/fer2013/test'

row, col = 48, 48
classes = 7

def count_exp(path, set_):
    dict_ = {}
    for expression in os.listdir(path):
        dir_ = path +"/" +expression
        dict_[expression] = len(os.listdir(dir_))
    df = pd.DataFrame(dict_, index=[set_])
    return df
train_count = count_exp(train_dir, 'train')
test_count = count_exp(test_dir, 'test')
print(train_count)
print(test_count)

# EDA 

# Plot of number of images in training set

In [4]:
train_count.transpose().plot(kind='bar')

# PLot of number of images in test set

In [5]:
test_count.transpose().plot(kind='bar')

In [6]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img
plt.figure(figsize=(14,22))
i = 1
for expression in os.listdir(train_dir):
    img = load_img((train_dir+"/" + expression + "/" + os.listdir(train_dir +"/"+ expression)[1]))
    plt.subplot(1,7,i)
    plt.imshow(img)
    plt.title(expression)
    plt.axis('off')
    i += 1
plt.show()

# Creating dataset 

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rotation_range=20,
                              rescale=1./255,
                              shear_range=0.1,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              width_shift_range=0.1,
                              height_shift_range=0.1)

training_data = train_gen.flow_from_directory(train_dir,
                                             target_size=(224,224),
                                             batch_size=64,
                                             color_mode = "grayscale",
                                             class_mode = "categorical")

In [8]:
valid_gen = ImageDataGenerator(rescale=1./255)

valid_data = valid_gen.flow_from_directory(test_dir,
                                          target_size=(224,224),
                                          batch_size=64,
                                          color_mode='grayscale',
                                          class_mode='categorical')

# Calling vgg model

In [9]:
from tensorflow.keras.applications.vgg19 import VGG19

vgg = VGG19(weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

# Creating Model

In [10]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model

input = Input(shape=(224,224,1))

conv = Conv2D(3, kernel_size=(3,3), padding='same')(input)

vgg = vgg(conv)
    
x = Flatten()(vgg)

pred = Dense(7, activation='softmax')(x)

model = Model(inputs=input, outputs=pred)

In [11]:
model.summary()

In [12]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)

model.compile(optimizer = opt,
              loss='categorical_crossentropy',
              metrics=['accuracy']
)

In [13]:
history = model.fit(training_data, epochs=100, validation_data = valid_data)

In [14]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [15]:
train_loss, train_acc = model.evaluate(training_data)
test_loss, test_acc = model.evaluate(valid_data)

In [17]:
from keras.preprocessing import image
test_img = image.load_img('../input/fer2013/test/angry/PrivateTest_13278552.jpg',target_size = (224,224),color_mode = "grayscale")
plt.imshow(test_img)

In [18]:
label_dict = {0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Neutral',5:'Sad',6:'Surprise'}

test_img = np.expand_dims(test_img,axis = 0)
test_img = test_img.reshape(1,224,224,1)
result = model.predict(test_img)
result = list(result[0])

img_index = result.index(max(result))
print(label_dict[img_index])

In [19]:
model.save('model_optimal.h5')

# Part2 - TESTING

## Installing depandancies

In [20]:
!pip install mtcnn

# Importing Required libraries

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
from mtcnn import MTCNN  # helps in detecting faces from image
from tensorflow.keras.models import load_model

In [23]:
model = load_model('./model_optimal.h5')
print('Model Loaded 100%')

In [25]:
img = 'https://static-bebeautiful-in.unileverservices.com/Unlock-flawless-skin_MobileHomeFeature.jpg'
try:
    image = plt.imread(img)
except SyntaxError:
    image = plt.imread(img, format='jpg')
plt.imshow(image)

# Detecting Face

In [26]:
detect = MTCNN()
results = detect.detect_faces(image)

x1, y1, width, height = results[0]['box']
x2, y2 = x1 + width, y1 + height

face = image[y1:y2, x1:x2]
plt.imshow(face)

# Resizing Image

In [27]:
test_image = np.resize(face, (224,224,1))

# Prediction

In [29]:
label_dict = {0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Neutral',5:'Sad',6:'Surprise'}

test_img = np.expand_dims(test_image, axis = 0)
test_img = test_img.reshape(-1,224,224,1)
result = model.predict(test_img)
result = list(result[0])

img_index = result.index(max(result))
emotion = label_dict[img_index]
print(emotion)

# Song Playing as per mood

In [31]:
from IPython.display import Audio
song_path = '../input/emotion-audio-files/Ringtones'
song = os.path.join(song_path,emotion)
print(f'Playing {emotion} song')


# <center>THE END</center>